In [ ]:
# run_path = "../../data/runs/model265-epoch3/"
# training_data = "../../data/processed/training.nc"
# training_data_mean = "../../data/processed/training.mean.nc"
# caseid = "control"

In [ ]:
from src.data.sam import SAMRun
from uwnet.thermo import vorcitity
import holoviews as hv
hv.extension('bokeh')
plt.style.use('colorblind')

%opts Image[width=600, height=400, colorbar=True](cmap='viridis')

def plot_2d(da, dynamic=False):
    time = da.time
    da = da.assign_coords(time=time-time[0])
    max = da.time.max('time')
    da = da.load().interp(time=np.arange(max))
    return hv.Dataset(da).to.image(["x", "y"], dynamic=dynamic).redim.unit(time="day")


In [ ]:
# Load the data
run = SAMRun(run_path, case=caseid)
ng = xr.open_dataset(training_data).isel(step=0)

# Run Namelist

In [ ]:
run.namelist

# 2D Fields

## Vorticity

In [ ]:
%%opts Image(cmap='RdBu_r')

d2d = run.data_2d
vort = vorcitity(d2d.U850, d2d.V850)
plot_2d(vort)\
.relabel("Relative Vort")\
.redim.range(Vorticity=(-1e-4, 1e-4))

## Precipitable Water

In [ ]:
%%opts Image
plot_2d(run.data_2d.PW)

## Zonal-mean PW vs time

In [ ]:
pw0 = run.data_2d.PW.mean('x')[0]
pw_anom = run.data_2d.PW.mean('x') - pw0
pw_anom[[10, 50, -1]].plot(hue='time')

## PW Eddy (from zonal mean)

In [ ]:
%%opts Image(cmap='RdBu_r')
pw_eddy = run.data_2d.PW- run.data_2d.PW.mean(['x', 'time'])
plot_2d(pw_eddy).redim.range(PW=(-20,20))\
.relabel("Zonal Anomaly of PW")

## P-E

In [ ]:
evap = ng.LHF / 2.51e6 * 86400
net_precip = ng.Prec - evap
net_precip.name='Net Precipitation'

In [38]:
try:
    net_precip_nn = -(run.data_3d.FQTNN * ng.layer_mass).sum('z')/1000
except AttributeError:
    net_precip_nn = run.data_2d.Prec - run.data_2d.LHF / 2.51e6*86400
truth_interp = net_precip.interp_like(net_precip_nn)

In [ ]:
plot_2d(net_precip_nn.rename("Net Precip"))

# Climate

Here I compute the mean climate of NG-Aqua and the NN+SAM bias (excluding the first 5 days).

In [ ]:
mean_ng = xr.open_dataset(training_data_mean)
mean_nn = run.data_3d.isel(time=slice(5,None)).mean(['x', 'time'])

In [ ]:
mean_ng

In [ ]:
def compare_climate(field):
    fig, axs = plt.subplots(1, 2, constrained_layout=True, figsize=(9, 3))
    mean_ng[field].plot.contourf(ax=axs[0], levels=11)
    
    bias = mean_nn[field] - mean_ng[field]
    bias.plot.contourf(ax=axs[1], levels=11)
    
    axs[0].set_title(f"A) NG-Aqua {field}")
    axs[1].set_title(f"B) NN Bias {field}")
    
    return axs

In [ ]:
compare_climate('QT');
compare_climate('SLI');
compare_climate('V');
compare_climate('U');

# Weather diagnostics

In [ ]:
def rms(x, dim='x'):
    return np.sqrt((x**2).mean(dim)) 

def corr(*args, dim='x'):
    x, y = [a-a.mean(dim) for a in args]
    sig2_x = (x**2).sum(dim)
    sig2_y = (y**2).sum(dim)
    return (x*y).sum(dim)/np.sqrt(sig2_x*sig2_y)

## P-E Pattern Correlation

### y vs time Hovmoller diagram

In [ ]:
patcor = corr(net_precip_nn, truth_interp)

patcor.plot(x='time')
patcor.plot.contour(x='time', color='k', levels=[.25, .5])
plt.title("Pattern Correlation of P-E")

## Meridional sector time series

Here, we divide the domain into 5 meridional sectors and plot the pattern correlation of P-E in both.

In [ ]:
def mean_lat_group(x, bins=5):
    dims =['x', 'y']
    return x.groupby_bins('y', bins=bins).mean(dims)

def corr_bins(*args, dim='x'):
    x, y = [a-a.mean(dim) for a in args]
    
    sig2_x = mean_lat_group(x**2)
    sig2_y = mean_lat_group(y**2)
    xy = mean_lat_group(x*y)
    return xy / np.sqrt(sig2_x * sig2_y)

In [ ]:
corrs = corr_bins(net_precip_nn, truth_interp)

In [ ]:
corrs['time'] = corrs.time - corrs.time[0]
corrs.plot(hue='y_bins')